In [ ]:
from pyspark.sql import SparkSession
import datetime
spark = SparkSession.builder \
    .appName("Streaming-Preparation")\
    .config("spark.drive.host","127.0.0.1") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.executor.memory","4g") \
    .config("spark.driver.memory","8g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
# spark.sparkContext.setLogLevel("DEBUG")
print(f'spark session created {spark} at {datetime.datetime.now()}')
spark.stop()
spark

spark session created <pyspark.sql.session.SparkSession object at 0x7d23dcfdfad0> at 2025-06-22 13:03:28.306384


In [95]:
''' This Application will read the data from the port 10000 that is opened by ncat web socket'''
# server_props = {
#     "host":"localhost",
#     "port":9999
# }
df0 = spark.readStream.format("socket").option("host","localhost").option("port",9992).load()

In [96]:
df0.printSchema()

root
 |-- value: string (nullable = true)



In [97]:
# Logic part
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# levit_txt ='./test_text.txt'
# df = spark.read.format("text").load(levit_txt)
# df=df.withColumn("words",F.split(F.col("value")," "))

df=df0.withColumn("words",F.split(F.col("value")," "))
df=df.withColumn("word",F.explode(F.col("words"))).drop("value","words")
results=df.groupBy("word").agg(F.count("word").alias("word_cnt")).orderBy("word")

# result.show()

In [ ]:
# WriteStream
results.writeStream.outputMode("complete").format("console").start().awaitTermination()
